# CS 5540 - Group 1 - Apache Spark Assignment

This is the submission document for our programming assignment over Apache Spark. 

The submission was written as a Jupyter notebook but will be exported to a PDF for submission. We can provide the GitHub repo or the original Jupyter notebook if requested.

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import col, split, explode

spark = SparkSession.builder\
    .appName("my-spark-app")\
    .config("spark.sql.catalogImplementation", "hive")\
    .getOrCreate()
sc = SparkContext.getOrCreate()

23/02/27 11:13:15 WARN Utils: Your hostname, Odais-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.161.4.110 instead (on interface en0)
23/02/27 11:13:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/27 11:13:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Table of Contents

1. [Team Members](#team-members)

## Team Members

This assignment was completed by the following team members (Group 1):

- Ayushman Das
- Koti Paruchuri
- Odai Athamneh
- Scott Brunton
- Varshith Thota

## Question 1

Question 1 is as follows: 

> Given file (`/data/shakespeare-1.txt`) contains the scenes from Shakespeare’s plays. You may use this 
file as an input dataset to identify the following notes for a student of Classical Drama.

In [3]:
# load the text file into a dataframe
df = spark.read.text("data/shakespeare-1.txt")

# split the text into words
df = df.select(split(col("value"), " ").alias("words"))

# explode the array of words into separate rows
df = df.select(explode(col("words")).alias("word"))

# show the first 5 words
df.show(5, truncate=False)


+-----+
|word |
+-----+
|This |
|is   |
|the  |
|100th|
|Etext|
+-----+
only showing top 5 rows



### Question 1.1

The question reads as follows:

> How many different countries are mentioned in the whole file? (Regardless of how many times a single country is mentioned, this country only contributes as a single entry). 

To address this question, we need a dataset of country names. There are many possible approaches to this problem, but we decided to use a dataset from [Kaggle](https://www.kaggle.com/datasets/fernandol/countries-of-the-world) that is sourced from the US government. This CSV file contains information on 227 present-day countries, including their names, population, and other information. We will use the `Country` column as our list of country names.

The caveat to this approach is that the dataset may not contain all countries, such as: 
- Countries that no longer exist
- Countries that are misspelled in the original Shakespearean text
- Countries where the name or spelling has changed over time

Addressing this issue is beyond the scope of this assignment and would likely require some degree of manual curation.

In [4]:
countries = spark.read.csv("data/countries.csv", header=True)
countries = countries.select("Country")
countries.show(5, truncate=False)

+---------------+
|Country        |
+---------------+
|Afghanistan    |
|Albania        |
|Algeria        |
|American Samoa |
|Andorra        |
+---------------+
only showing top 5 rows



Now that we have our list of countries, we can use a simple `.join()` to find the number of countries mentioned in the Shakespearean text. We will use the `Country` column as our key and perform an inner join with the Shakespearean text. This will return a new DataFrame with only the rows that have a match in both DataFrames. We can then use `.count()` to get the number of rows in the resulting DataFrame.

In [14]:
df.join(countries, df.word == countries.Country).show(5, truncate=False)

+----+-------+
|word|Country|
+----+-------+
+----+-------+



In [15]:
output = []

for country in countries.collect():
    print(country.Country)

    if df.filter(df.word == country.Country).count() > 0:
        output.append(country.Country)
        
output

Afghanistan 
Albania 
Algeria 
American Samoa 
Andorra 
Angola 
Anguilla 
Antigua & Barbuda 
Argentina 
Armenia 
Aruba 
Australia 
Austria 
Azerbaijan 
Bahamas, The 
Bahrain 
Bangladesh 
Barbados 
Belarus 
Belgium 
Belize 
Benin 
Bermuda 
Bhutan 
Bolivia 
Bosnia & Herzegovina 
Botswana 
Brazil 
British Virgin Is. 
Brunei 
Bulgaria 
Burkina Faso 
Burma 
Burundi 
Cambodia 
Cameroon 
Canada 
Cape Verde 
Cayman Islands 
Central African Rep. 
Chad 
Chile 
China 
Colombia 
Comoros 
Congo, Dem. Rep. 
Congo, Repub. of the 
Cook Islands 
Costa Rica 
Cote d'Ivoire 
Croatia 
Cuba 
Cyprus 
Czech Republic 
Denmark 
Djibouti 
Dominica 
Dominican Republic 
East Timor 
Ecuador 
Egypt 
El Salvador 
Equatorial Guinea 
Eritrea 
Estonia 
Ethiopia 
Faroe Islands 
Fiji 
Finland 
France 
French Guiana 
French Polynesia 
Gabon 
Gambia, The 
Gaza Strip 
Georgia 
Germany 
Ghana 
Gibraltar 
Greece 
Greenland 
Grenada 
Guadeloupe 
Guam 
Guatemala 
Guernsey 
Guinea 
Guinea-Bissau 
Guyana 
Haiti 
Honduras 
Hong Kon

[]